In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import folium
from selenium import webdriver as wb
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
import time

In [ ]:
import pandas as pd
import numpy as np

# 예제 데이터프레임 생성
df = pd.DataFrame({
    'A': [1, 2, 3],
    'B': [4, 5, 6]
})

# 추가할 배열 생성 (각 원소가 배열인 리스트)
new_column = [[1,2],[3,4],[5,6]]

# 새로운 열 'C'를 추가하고 배열을 설정
df['C'] = new_column
for i in df.loc[0,"C"]:
    print(i)

In [ ]:
a=pd.DataFrame([[1,2,3],[4,5,6]])
a.index()

In [ ]:
df = pd.DataFrame({
    'A': ['foo', 'bar', 'baz'],
    'B': ['one', 'two', 'three'],
    'C': [1, 2, 3]
})



df.index=['a','q','w']
df.loc['w','B']

In [ ]:
NS_DIFF_1M = 0.000007
EW_DIFF_1M = 0.000009
PATROL_HR = 900
GRID_DIFF = 90
data_path = "data/"
encoding = "utf-8-sig"
#상계1
police = np.array([37.679699, 127.055066])

In [ ]:
def filtering(police, grid):
    ft = (grid - police).copy()
    ft[:,0] /= NS_DIFF_1M
    ft[:,1] /= EW_DIFF_1M
    ft = np.sqrt( np.sum( np.square(ft), axis=1 ))
    grid = grid[ft <= PATROL_HR + 10, :]
    return grid

In [ ]:
def grids(police):
    hcount = PATROL_HR//GRID_DIFF
    ew = np.arange(-hcount, hcount + 1, dtype=np.float64)
    ns = ew.reshape(-1,1).copy()
    ew *= EW_DIFF_1M * GRID_DIFF
    ns *= NS_DIFF_1M * GRID_DIFF
    
    result = np.full((2*hcount+1, 2*hcount+1, 2), police, dtype=np.float64)
    result[:,:,0] += ns
    result[:,:,1] += ew
    
    return filtering(police, result.reshape(-1,2))

In [ ]:
def draw_marker(m, grid):
    for i in grid:
        folium.CircleMarker(location=i, radius=10, color='#000000').add_to(m)

In [ ]:
m = folium.Map(location = police, zoom_start=60)
folium.Marker(location=police, 
              popup='지구대',
              icon=folium.Icon(color='lightblue', icon='star')
).add_to(m)

In [ ]:
police_grid = grids(police)
draw_marker(m, police_grid)

In [ ]:
m

In [ ]:
#police_grid_data = pd.DataFrame(police_grid, columns=["위도", "경도"])
#police_grid_data.to_csv("상계1위경도.csv", index=False, encoding=encoding)

In [ ]:
police_grid_data=pd.read_csv(data_path+"상계1주소.csv", encoding=encoding)
police_grid_data["유형"] = None
police_grid_data

In [ ]:
DELAY_TIME = 1
map_url = "https://map.kakao.com/"

In [ ]:
def openweb(mode="window"):
    if mode == "no":
        options = wb.ChromeOptions()
        options.add_argument('headless')
        driver = wb.Chrome(options=options)
    else:
        driver = wb.Chrome()
    driver.get(map_url)
    time.sleep(DELAY_TIME)
    return driver

In [ ]:
def search(browser, address):
    search_box = browser.find_element(By.ID, "search.keyword.query")
    while search_box.get_attribute("value"):
        search_box.send_keys(Keys.CONTROL, 'a')
        search_box.send_keys(Keys.BACKSPACE)
    search_box.send_keys(address)
    search_box.send_keys(Keys.RETURN)
    time.sleep(DELAY_TIME)

In [ ]:
def find_postage(browser, num_addr):
    if "산" in num_addr:
        return "nozip"
    search(browser, num_addr)
    try:
        postage = browser.find_element(By.CLASS_NAME, "zip")
    except NoSuchElementException:
        return "nozip"
    else:
        return postage.text
    
def find_build_name(browser, road_addr, num_addr):
    search(browser, road_addr)
    try:
        buildname = browser.find_element(By.CLASS_NAME, "building")
    except NoSuchElementException:
        return find_postage(browser, num_addr)
    else:
        return buildname.text

In [ ]:
#browser = openweb("no")

In [ ]:
#for i in police_grid_data.index:
    #police_grid_data.iloc[i, 4] = find_build_name(browser, police_grid_data.iloc[i, 3], police_grid_data.iloc[i, 2])

In [ ]:
#police_grid_data.to_csv(data_path+"유형.csv",encoding=encoding)

In [ ]:
import pandas as pd

# 데이터를 담은 리스트
data = [[1, 1], [1, 2], [1, 3]]

# 인덱스를 튜플로 구성하여 MultiIndex 생성
index = pd.MultiIndex.from_tuples([(1, 1), (1, 2), (1, 3)], names=['outer', 'inner'])

# 데이터프레임 생성
df = pd.DataFrame(data, index=index, columns=['A', 'B'])

# 결과 출력
df

In [ ]:
import pandas as pd

# 기존의 DataFrame
df = pd.DataFrame({'A': [1, 2, 3], 'B': [4, 5, 6]}, index=['1', '2', '3'])

# MultiIndex 생성
multi_index = pd.MultiIndex.from_tuples([(1, 1), (1, 2), (1, 3)], names=['outer', 'inner'])

# MultiIndex를 지정하여 데이터 추가
df = pd.DataFrame([[1], [2], [3]], index=multi_index, columns=['C']).combine_first(df)

# 결과 출력
df

In [ ]:
import pandas as pd

# 기존의 DataFrame
df = pd.DataFrame({'A': [1, 2, 3], 'B': [4, 5, 6]}, index=['1', '2', '3'])

# MultiIndex 생성
multi_index = pd.MultiIndex.from_tuples([(1, 1), (1, 2), (1, 3)], names=['outer', 'inner'])

# 기존 DataFrame의 인덱스를 MultiIndex로 변환
df.index = pd.MultiIndex.from_tuples([(i,) for i in df.index], names=['outer'])

# 새로운 DataFrame 생성
new_df = pd.DataFrame([[1], [2], [3]], index=multi_index, columns=['C'])

# 기존 DataFrame과 새로운 DataFrame 결합
result_df = new_df.combine_first(df)

# 결과 출력
result_df

In [ ]:
import numpy as np
import pandas as pd
a=pd.DataFrame([1,2])
print(a)
a.iloc[0,1]

In [ ]:
a[["v","b"]]

In [ ]:
b=np.array([[0,0,0],[0,0,0]])
b

In [ ]:
b+a.values

In [ ]:
v=pd.DataFrame([[1,2,3],[4,5,6]])
v+(b+a)

In [ ]:
def filtering(police, data):
    ft = (data - police).copy()
    ft[:,0] /= NS_DIFF_1M
    ft[:,1] /= EW_DIFF_1M
    ft = np.sqrt( np.sum( np.square(ft), axis=1 ))
    data = data[ft < PATROL_HR + GRID_DIFF/2, :]
    return data